In [1]:
from utils.models import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

## Load

In [2]:
run = 4

In [3]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
        "train_data/total.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=12,
    shuffle=True,
)

In [4]:
import torch
import torch.nn as nn
import torch.optim as opt

In [5]:
torch.manual_seed(2)

In [6]:
# model = NeuFM(
#     GMF(users, movies, 8),
#     MLP(users, movies, 16, [16, 8]),
#     alpha=0.5,
# ).cuda()

model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

model

NeuFM(
  (h): Linear(in_features=24, out_features=1, bias=False)
  (gmf): GMF(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (h): Identity()
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Identity()
  )
)

## Train

In [7]:
criterion = nn.MSELoss().cuda()
optimizer = opt.Adam(model.parameters(), lr=0.0005)

prev_train_loss = math.inf
for epoch in tqdm(range(20)):
    n_batches = len(train_dataloader)
    avg_loss = 0
    
    # Train step
    for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        optimizer.zero_grad()
        y_hat = model(vus, vis)
        
        loss = criterion(y_hat, rs)
        loss.backward()
        optimizer.step()
        
        avg_loss += math.sqrt(float(loss.detach().cpu()))
                
    train_loss = avg_loss/n_batches
    print(f"epoch: {epoch}, train_loss: {train_loss}")
    
    if train_loss < prev_train_loss:
        prev_train_loss = train_loss
        torch.save(
            model.state_dict(),
            f"/home/nubol23/Documents/NCF_weights/run_{run}/final-{epoch}-{train_loss}.pt"
        )

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 0, train_loss: 2.265340065619825
epoch: 1, train_loss: 0.9391735815352601
epoch: 2, train_loss: 0.9012580229718545
epoch: 3, train_loss: 0.882264369917544
epoch: 4, train_loss: 0.8699250860606071
epoch: 5, train_loss: 0.8614463894519964
epoch: 6, train_loss: 0.8551574642340174
epoch: 7, train_loss: 0.8497990895704707
epoch: 8, train_loss: 0.8443028633563981
epoch: 9, train_loss: 0.838152273060414
epoch: 10, train_loss: 0.8315161017284126
epoch: 11, train_loss: 0.8240868726916466
epoch: 12, train_loss: 0.8166196537907313
epoch: 13, train_loss: 0.8084401909959371
epoch: 14, train_loss: 0.7998390728808266
epoch: 15, train_loss: 0.7906841245973248
epoch: 16, train_loss: 0.7812810769141403
epoch: 17, train_loss: 0.7724331434234724
epoch: 18, train_loss: 0.7641060107479849
epoch: 19, train_loss: 0.7567447456755086


## Predict

In [8]:
trained_weights = "final-19-0.7567447456755086.pt"

In [9]:
# trained_model = NeuFM(
#     GMF(users, movies, 8),
#     MLP(users, movies, 16, [16, 8]),
#     alpha=0.5,
# )

trained_model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

trained_model.load_state_dict(
    torch.load(
        f"/home/nubol23/Documents/NCF_weights/run_{run}/{trained_weights}"
    )
)
trained_model.cuda().eval()

NeuFM(
  (h): Linear(in_features=24, out_features=1, bias=False)
  (gmf): GMF(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (h): Identity()
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Identity()
  )
)

In [10]:
test_dataloader = DataLoader(
    RatingsDataset(
        "train_data/test.csv",
        "user_id",
        "movie_id",
    ), 
    batch_size=1024,
    num_workers=12
)

In [11]:
test_predictions = []

for vus, vis in test_dataloader:
    vus = vus.cuda()
    vis = vis.cuda()

    pred = torch.clip(trained_model(vus, vis), 1, 5).cpu().ravel().tolist()
    test_predictions += pred

In [12]:
import pandas as pd

test_csv = pd.read_csv("../../data/test_data.csv")

In [13]:
out_df = pd.DataFrame.from_dict(
    {
        "id": list(test_csv["id"]),
        "rating": test_predictions
    }
)

out_df.head()

,id,rating
0,0,3.051610
1,1,3.293595
2,2,2.713303
3,3,3.236285
4,4,3.643629


In [14]:
out_df.to_csv(f"outputs_csv/neumf_6.csv", index=False)